### Introduction and Schema Diagram

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Overview of the Data
* running below command to see all tables and views in database

In [3]:
%%sql
SELECT
  name,
  type
FROM sqlite_master
WHERE type IN ("table", "view");


Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
%%sql
SELECT *
FROM track
limit 5;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [5]:
%%sql
SELECT *
FROM invoice_line
limit 5;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


### Selecting New Albums to Purchase¶
##### Data needs for the query
- need track_id, quantity, invoice_id from invoice_line
- need track_id, genre_id from track
- need genre_id, name from genre
- need country_id, invoice_id from invoice

In [6]:
%%sql
CREATE view genre_country AS
  SELECT 
    il.track_id,
    il.quantity,
    il.invoice_id,
    t.genre_id,
    g.name,
    i.billing_country
  FROM invoice_line il
  INNER JOIN invoice i on i.invoice_id = il.invoice_id
  INNER JOIN track t ON t.track_id = il.track_id
  INNER JOIN genre g ON g.genre_id = t.genre_id

    
  

(sqlite3.OperationalError) table genre_country already exists
[SQL: CREATE view genre_country AS
  SELECT 
    il.track_id,
    il.quantity,
    il.invoice_id,
    t.genre_id,
    g.name,
    i.billing_country
  FROM invoice_line il
  INNER JOIN invoice i on i.invoice_id = il.invoice_id
  INNER JOIN track t ON t.track_id = il.track_id
  INNER JOIN genre g ON g.genre_id = t.genre_id]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [7]:
%%sql
select *
from genre_country
limit 5;

Done.


track_id,quantity,invoice_id,genre_id,name,billing_country
1158,1,1,1,Rock,USA
1159,1,1,1,Rock,USA
1160,1,1,1,Rock,USA
1161,1,1,1,Rock,USA
1162,1,1,1,Rock,USA


In [8]:
%%sql
SELECT 
  SUM(quantity) total_tracks,
  ROUND(CAST(SUM(quantity)as float) / CAST((select count(*) from genre_country)as float),4) percentage,
  name genre,
  billing_country country
FROM genre_country
GROUP BY genre, country
HAVING country = 'USA'
ORDER BY total_tracks DESC

Done.


total_tracks,percentage,genre,country
561,0.1179,Rock,USA
130,0.0273,Alternative & Punk,USA
124,0.0261,Metal,USA
53,0.0111,R&B/Soul,USA
36,0.0076,Blues,USA
35,0.0074,Alternative,USA
22,0.0046,Latin,USA
22,0.0046,Pop,USA
20,0.0042,Hip Hop/Rap,USA
14,0.0029,Jazz,USA


### Genre Conclusion - Which Three Artists to Choose
From the above results it is suggested that based on best selling genres in the USA, that the artists Red Tone, Meteor and the Girls, and Slim Jim Bites be selected.

### Analyzing Employee Sales Performance
##### Data Needs
- employee id, last_name, first_name, title, hire_date, reports_to from employee
- support_rep_id, customer_id from customer
- customer_id, total from invoice

In [9]:
%%sql
SELECT
 e1.employee_id employee_id,
 e1.first_name || ' ' || e1.last_name employee_name,
 e1.title title,
 e1.hire_date hire_date,
 e1.reports_to reports_to,
 e2.first_name || ' ' || e2.last_name  supervisor_name,
 c.support_rep_id sales_rep,
 c.customer_id cust_id,
 SUM(i.total) total_sales
from employee e1
LEFT JOIN employee e2 on e1.reports_to = e2.employee_id
INNER JOIN customer c on c.support_rep_id = e1.employee_id
INNER JOIN invoice i on i.customer_id = c.customer_id
GROUP BY employee_name
ORDER BY 9 DESC
;

Done.


employee_id,employee_name,title,hire_date,reports_to,supervisor_name,sales_rep,cust_id,total_sales
3,Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,2,Nancy Edwards,3,59,1731.510000000004
4,Margaret Park,Sales Support Agent,2017-05-03 00:00:00,2,Nancy Edwards,4,56,1584.0000000000032
5,Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,2,Nancy Edwards,5,57,1393.9200000000028


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

### Analyzing Sales by Country

- customer - customer_id, country
- invoice - total, invoice_id, customer_id


In [29]:
%%sql
SELECT 
  COUNT(distinct c.customer_id) num_customers,
  c.country,
  ROUND(SUM(i.total), 2) total_sales,
  COUNT(i.invoice_id) total_orders,
  ROUND(SUM(i.total) / COUNT(c.customer_id),2) sales_per_cust,
  ROUND(SUM(i.total) / COUNT(i.invoice_id),2) avg_order_value
FROM customer c
INNER JOIN invoice i on i.customer_id = c.customer_id
GROUP BY country
ORDER BY 3 DESC
limit 125;

Done.


num_customers,country,total_sales,total_orders,sales_per_cust,avg_order_value
13,USA,1040.49,131,7.94,7.94
8,Canada,535.59,76,7.05,7.05
5,Brazil,427.68,61,7.01,7.01
5,France,389.07,50,7.78,7.78
4,Germany,334.62,41,8.16,8.16
2,Czech Republic,273.24,30,9.11,9.11
3,United Kingdom,245.52,28,8.77,8.77
2,Portugal,185.13,29,6.38,6.38
2,India,183.15,21,8.72,8.72
1,Ireland,114.84,13,8.83,8.83


In [28]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


Based on the data, there may be opportunity in the following countries:
- Czech Republic
- United Kingdom
- India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

### Albums vs Individual Tracks

In [30]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.